# ProfessorAI
Ask questions and get answers based on lecture content.

In [20]:
import ipywidgets as widgets
import langchain
import pinecone
import openai
import tiktoken
import os

In [21]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain

from langchain.memory import ConversationBufferMemory

from langchain.vectorstores import Pinecone
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

# Don't forget to set your API keys

In [22]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

# You only need to set these environment variables if you want to use the Pinecone Vectorstorage
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
PINCONE_API_ENV = os.environ.get("PINECONE_API_ENV")

# ^ ^ ^

In [23]:
embeddings_engine = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [24]:
#pinecone.init(
#    api_key=PINECONE_API_KEY,
#    environment=PINCONE_API_ENV
#)
#index_name = "diskstrukt2023"

In [25]:
#db = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings_engine)
db = Chroma(persist_directory="../transcription_db", embedding_function=embeddings_engine)

Select what AI model to use to get your answer here:

In [26]:
# Valid model names as of making are
    #"gpt-4"
    #"gpt-4-0314"
    #"gpt-4-0613"
    #"gpt-4-32k"
    #"gpt-4-32k-0314"
    #"gpt-4-32k-0613"
    #"gpt-3.5-turbo"
    #"gpt-3.5-turbo-0301"
    #"gpt-3.5-turbo-0613"
    #"gpt-3.5-turbo-16k"
    #"gpt-3.5-turbo-16k-0613"
    #"text-ada-001"
    #"ada"
    #"text-babbage-001"
    #"babbage"
    #"text-curie-001"
    #"curie"
    #"davinci"
    #"text-davinci-003"
    #"text-davinci-002"
    #"code-davinci-002"
    #"code-davinci-001"
    #"code-cushman-002"
    #"code-cushman-001"
model = "gpt-3.5-turbo"

#llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY, model=model, streaming=True)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
conversational_qa_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=db.as_retriever(), memory=memory)

## Gradio UI

In [ ]:
import gradio as gr

def answer_question(question):
    question_result = conversational_qa_chain({"question": question})
    return question_result["answer"]

QA_Interface = gr.Interface(fn=answer_question, 
    inputs="text", 
    outputs="text",
    layout="vertical"
    )

with gr.Blocks() as chat_ui:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        bot_message = answer_question(message)
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

chat_ui.launch()

